In [13]:
import pandas as pd 
import numpy as np
ingr_ns=pd.read_csv('tabellen/data_ingr.csv')
ingr_schoon=pd.read_csv('tabellen/data_ingr_clean.csv') 
recipes = pd.read_csv('tabellen/Recepten_kiezen_gescheiden.csv')
recipes

,Unnamed: 0,title,persons,time,calories,stars,url,image
0,0,Courgette carbonara,4,20,575,0,https://www.ah.nl/allerhande/recept/R-R1188954...,https://static.ah.nl/static/recepten/img_RAM_P...
1,0,Mexicaanse taco's met vis en zoete aardappel,4,20,655,0,https://www.ah.nl/allerhande/recept/R-R1188828...,https://static.ah.nl/static/recepten/img_09292...
2,0,Gegrilde-paprikasalade met feta en munt,4,15,205,0,https://www.ah.nl/allerhande/recept/R-R1188715...,https://static.ah.nl/static/recepten/img_09094...
3,0,Viscurry met zelfgemaakte currypasta,4,20,345,0,https://www.ah.nl/allerhande/recept/R-R1187967...,https://static.ah.nl/static/recepten/img_08598...
4,0,Spruitjesgratin met zoete aardappel,4,10,665,0,https://www.ah.nl/allerhande/recept/R-R1187741...,https://static.ah.nl/static/recepten/img_08516...
...,...,...,...,...,...,...,...,...
115,0,Rodekoolsteak met blauwaderkaas en truffel,4,15,550,0,https://www.ah.nl/allerhande/recept/R-R1189726...,https://static.ah.nl/static/recepten/img_10034...
116,0,Vegan wraps met tofu en rodekoolsalade,4,20,670,0,https://www.ah.nl/allerhande/recept/R-R1195491...,https://static.ah.nl/static/recepten/img_RAM_P...
117,0,Hachee uit de oven,4,15,365,0,https://www.ah.nl/allerhande/recept/R-R1193014...,https://static.ah.nl/static/recepten/img_RAM_P...
118,0,Aardappel en zuurkool,4,15,405,0,https://www.ah.nl/allerhande/recept/R-R758540/...,https://static.ah.nl/static/recepten/img_00767...


In [17]:
import csv
with open('tabellen/Recepten_kiezen_f.csv', newline='') as csvfile:
    csvlezer = csv.reader(csvfile, delimiter=',', quotechar='"')
    for x in csvlezer:
        x = recipes
recipes[117:123]

recipes

,Unnamed: 0,title,persons,time,calories,stars,url,image
0,Courgette carbonara,4,20,575,0,https://www.ah.nl/allerhande/recept/R-R1188954...,https://static.ah.nl/static/recepten/img_RAM_P...,NaN
0,Mexicaanse taco's met vis en zoete aardappel,4,20,655,0,https://www.ah.nl/allerhande/recept/R-R1188828...,https://static.ah.nl/static/recepten/img_09292...,NaN
0,Gegrilde-paprikasalade met feta en munt,4,15,205,0,https://www.ah.nl/allerhande/recept/R-R1188715...,https://static.ah.nl/static/recepten/img_09094...,NaN
0,Viscurry met zelfgemaakte currypasta,4,20,345,0,https://www.ah.nl/allerhande/recept/R-R1187967...,https://static.ah.nl/static/recepten/img_08598...,NaN
0,Spruitjesgratin met zoete aardappel,4,10,665,0,https://www.ah.nl/allerhande/recept/R-R1187741...,https://static.ah.nl/static/recepten/img_08516...,NaN
...,...,...,...,...,...,...,...,...
0,Rodekoolsteak met blauwaderkaas en truffel,4,15,550,0,https://www.ah.nl/allerhande/recept/R-R1189726...,https://static.ah.nl/static/recepten/img_10034...,NaN
0,Vegan wraps met tofu en rodekoolsalade,4,20,670,0,https://www.ah.nl/allerhande/recept/R-R1195491...,https://static.ah.nl/static/recepten/img_RAM_P...,NaN
0,Hachee uit de oven,4,15,365,0,https://www.ah.nl/allerhande/recept/R-R1193014...,https://static.ah.nl/static/recepten/img_RAM_P...,NaN
0,Aardappel en zuurkool,4,15,405,0,https://www.ah.nl/allerhande/recept/R-R758540/...,https://static.ah.nl/static/recepten/img_00767...,NaN


In [10]:
onze_ns = ingr_ns[ingr_ns['recipe'].isin(recipes['title'])]

In [11]:
onze_schoon = ingr_schoon[ingr_schoon['recipe'].isin(recipes['title'])]

In [12]:
class Recepten: 
    def __init__(self, df): 
        self.data_frame = df
        

In [13]:
recept_schoon = Recepten(ingr_schoon)

print(recept_schoon.data_frame)

       Unnamed: 0                            recipe            ingredient  \
0               0     Kruidnoten met choco-discodip         melkchocolade   
1               1     Kruidnoten met choco-discodip            kruidnoten   
2               2     Kruidnoten met choco-discodip              discodip   
3               3           Kruidnoten in marsepein       blankemarsepein   
4               4           Kruidnoten in marsepein  ongezouten roomboter   
...           ...                               ...                   ...   
49814       71804  Biefstuk met rodewijnsaus en ham             olijfolie   
49815       71805  Biefstuk met rodewijnsaus en ham           biefstukken   
49816       71806  Biefstuk met rodewijnsaus en ham                 boter   
49817       71807  Biefstuk met rodewijnsaus en ham            serranoham   
49818       71808  Biefstuk met rodewijnsaus en ham             rode wijn   

       quantity unit  
0         100.0    g  
1         100.0    g  
2     

In [14]:
recept_ns = Recepten(ingr_ns)

print(recept_ns.data_frame)

       Unnamed: 0                            recipe            ingredient  \
0               0     Kruidnoten met choco-discodip         melkchocolade   
1               1     Kruidnoten met choco-discodip            kruidnoten   
2               2     Kruidnoten met choco-discodip              discodip   
3               3           Kruidnoten in marsepein      blanke marsepein   
4               4           Kruidnoten in marsepein  ongezouten roomboter   
...           ...                               ...                   ...   
49814       71804  Biefstuk met rodewijnsaus en ham             olijfolie   
49815       71805  Biefstuk met rodewijnsaus en ham           biefstukken   
49816       71806  Biefstuk met rodewijnsaus en ham                 boter   
49817       71807  Biefstuk met rodewijnsaus en ham            serranoham   
49818       71808  Biefstuk met rodewijnsaus en ham             rode wijn   

       quantity     unit  
0         100.0        g  
1         100.0      

In [15]:
recipes_schoon = recipes.dropna(subset=['title'])
recipes_schoon

,Unnamed: 0,title,persons,time,calories,stars,url,image
0,Courgette carbonara,4,20,575,0,https://www.ah.nl/allerhande/recept/R-R1188954...,https://static.ah.nl/static/recepten/img_RAM_P...,NaN
0,Mexicaanse taco's met vis en zoete aardappel,4,20,655,0,https://www.ah.nl/allerhande/recept/R-R1188828...,https://static.ah.nl/static/recepten/img_09292...,NaN
0,Gegrilde-paprikasalade met feta en munt,4,15,205,0,https://www.ah.nl/allerhande/recept/R-R1188715...,https://static.ah.nl/static/recepten/img_09094...,NaN
0,Viscurry met zelfgemaakte currypasta,4,20,345,0,https://www.ah.nl/allerhande/recept/R-R1187967...,https://static.ah.nl/static/recepten/img_08598...,NaN
0,Spruitjesgratin met zoete aardappel,4,10,665,0,https://www.ah.nl/allerhande/recept/R-R1187741...,https://static.ah.nl/static/recepten/img_08516...,NaN
...,...,...,...,...,...,...,...,...
0,Rodekoolsteak met blauwaderkaas en truffel,4,15,550,0,https://www.ah.nl/allerhande/recept/R-R1189726...,https://static.ah.nl/static/recepten/img_10034...,NaN
0,Vegan wraps met tofu en rodekoolsalade,4,20,670,0,https://www.ah.nl/allerhande/recept/R-R1195491...,https://static.ah.nl/static/recepten/img_RAM_P...,NaN
0,Hachee uit de oven,4,15,365,0,https://www.ah.nl/allerhande/recept/R-R1193014...,https://static.ah.nl/static/recepten/img_RAM_P...,NaN
0,Aardappel en zuurkool,4,15,405,0,https://www.ah.nl/allerhande/recept/R-R758540/...,https://static.ah.nl/static/recepten/img_00767...,NaN


In [16]:
a= np.array([recipes_schoon['title']]) 
a 

array([[ 4,  4,  4,  4,  4,  4,  4,  4,  4, 12,  4,  4,  4,  4,  1,  4,
        20, 24, 20,  4,  4,  4,  4,  4,  4, 10, 16,  4,  4,  2,  4, 12,
         4,  8,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,  4,
         4,  4,  4,  4,  4,  4,  4, 12,  4, 20,  4, 12, 12,  4,  4,  4,
         4,  4,  4,  8,  4,  6, 12,  4,  4,  8,  4,  4,  4,  2,  4,  4,
         4,  2,  4,  2,  4,  4,  4,  1,  2,  4,  4,  4,  4,  4,  4,  4,
         8,  4, 12,  4,  4, 20,  2,  4,  4,  4,  4,  4,  4,  4,  4,  4,
         4,  4,  4,  4,  4,  4,  4,  4]])

In [32]:
set_ingr= { ingr_schoon['ingredient'][i] for i in range(0, len(ingr_schoon)) if ingr_schoon['recipe'][i] in a}
set_ingr 

{'100% notenpasta amandel',
 'AH BASIC spaghetti',
 'AH Biologisch carpaccio met kaas, dressing en pijnboompitten',
 'Aardappel',
 'BYB red cheddar',
 'Balsajo zwarte knoflook',
 'Chioggia biet',
 'Country Cow Cheese slices',
 'De Zaanse Hoeve ongezouten roomboter',
 'De Zaanse Hoeve verse slagroom',
 'Euroma Original Spices Thai green curry',
 'Gelderse rookworst',
 'Go-Tan volkorennoedels',
 'Goudse belegen komijnekaas 48+',
 'Grand’Italia Sugocasa Erbe in fles',
 'Grand’Italia gnocchi',
 'HAK linzen in zak',
 'Hollandse bruine bonen uit pot',
 'Italiaanse roerbakgroente champignons',
 'Kaas',
 'Kipfilet',
 'Kool',
 'Kruidenmix tex mex',
 'Kühne mierikswortel in pot',
 'Peen',
 'Zalm',
 'ahornsiroop',
 'amandeldrink',
 'amandelen dry roasted',
 'amandelmeel',
 'amandelschaafsel',
 'amandelspijs',
 'ananas',
 'ansjovisfilets in olie in blik',
 'appel',
 'appelciderazijn',
 'appelmoes 0%',
 'arachideolie',
 'aspergebroccoli',
 'aubergine',
 'augurkenblokjes',
 'azijn',
 'babyromainesla